# it`s not official just use it as a test#

In [47]:
import camelot
from dotenv import load_dotenv
import arabic_reshaper
from bidi.algorithm import get_display
from joblib import Memory
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
from pytesseract import Output
import os
import cv2
import numpy as np

In [48]:

# Define a directory for cache storage
memory = Memory("./cache_dir", verbose=0)


In [49]:
@memory.cache
def get_pdf_tables(pdf_docs):
    pdfs_reader = []
    for pdf in pdf_docs:
        pdf_reader = camelot.read_pdf(pdf,pages='1',) #address of pdf file(pdf)
        pdfs_reader.append(pdf_reader)
    return pdfs_reader



def fix_arabic_text(cell): # Fix Arabic text direction
    try:
        reshaped_text = arabic_reshaper.reshape(cell)  # Reshape Arabic characters
        bidi_text = get_display(reshaped_text)  # Apply BiDi algorithm
        return bidi_text
    except Exception:
        return cell  # Return as is if not Arabic
    

In [50]:
directory_path = r".\documents"

pdf_files = []
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.pdf'):
            pdf_files.append(os.path.join(root, file))

# Print the list of PDF files with their paths
print("PDF files found:")
for pdf_file in pdf_files:
    print(pdf_file)


PDF files found:
.\documents\الاثنين 16-12-2024 د19 القاهرة الجديدة.pdf
.\documents\الاثنين جلسة 1-1-2024 د19.pdf
.\documents\الاثنين جلسة 1-4-2024 د19 وراثات.pdf
.\documents\د(19)اسرة السيدة زينب 22-12-2024.pdf
.\documents\د(20)اسرة باب الشعرية 19-12-2024.pdf
.\documents\د(20)اسرة عابدين18-12-2024.pdf
.\documents\د(22) اسرة  الموسكى  جلسة 21-12-2024.pdf
.\documents\ورثات الزاوية 16-12.pdf
.\documents\ورثات الشرابية 17-12.pdf


In [51]:
pdfs_tables = get_pdf_tables(pdf_files)

print(pdfs_tables)


[<TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>, <TableList n=1>]


In [52]:
def get_head_from_first_page(pdf): # Get the head of the table from the first page only of each PDF
        first_page = pdf[0]
        first_page.df = first_page.df.map(fix_arabic_text)
        # Access the table's structure
        cells = first_page.cells  # List of all detected cells
        head = f"{cells[0][0].x1}, {cells[0][0].y2}, {cells[0][-1].x2}, {cells[0][0].y1}"
        return head
head_coordinates = {}
for i in range (len(pdfs_tables)):
    print(i)
    head_coordinates[i]=get_head_from_first_page(pdfs_tables[i])
print(head_coordinates)

0
1
2
3
4
5
6
7
8
{0: '19.921338334108565, 822.2581242873432, 575.1986688180718, 619.2588369441278', 1: '19.92133879190301, 822.2581242873432, 575.1986688180718, 638.220638540479', 2: '19.91383874546188, 822.2581242873432, 575.1986688180718, 664.1431014823261', 3: '19.92133879190301, 822.2581242873432, 575.1986688180718, 619.2588369441278', 4: '19.92133741851967, 822.2581242873432, 575.1986688180718, 619.2588369441278', 5: '19.92133879190301, 822.2581242873432, 575.1986688180718, 619.2588369441278', 6: '19.92133741851967, 822.2581242873432, 575.1986688180718, 619.2588369441278', 7: '19.921324600275128, 822.2581242873432, 575.1986688180718, 664.1431014823261', 8: '19.92122205431878, 822.2581242873432, 575.1986688180718, 664.1431014823261'}


## You don`t need to run this cell


In [53]:
%%script false
def check_on_pdf_grid(pdf_path,key):
    file_name =os.path.basename(pdf_path)
    nested_tables = camelot.read_pdf(pdf_path, pages='1',flavor='stream',table_areas=[head_coordinates[key]],edge_tol=500,)
    camelot.plot(table=nested_tables[0], kind="grid")
    
for pdf_path,key in zip(pdf_files,head_coordinates):
    print(pdf_path,key)
    check_on_pdf_grid(pdf_path,key)

Couldn't find program: 'false'


In [54]:
#get the first page of each pdf as an image
pages_images = []
for pdf_path,key in zip(pdf_files,head_coordinates):
    # Convert the PDF page to an image
    pages = convert_from_path(pdf_path, first_page=1, last_page=1, dpi=300,poppler_path=r".\libs\poppler-24.08.0\Library\bin")
    pages_images.append(pages[0])
    
print(pages_images)

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1EB60>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1ED70>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1EB30>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1F850>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1F190>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1F010>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1EFB0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1F7F0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509 at 0x194EAE1E830>]


In [55]:
#convert head_coordinates to list of int
head_coordinates_int = {}
for key in head_coordinates:
    head_coordinates_int[key] = [int(float(coord)) for coord in head_coordinates[key].split(",")]
    
print(head_coordinates_int)

{0: [19, 822, 575, 619], 1: [19, 822, 575, 638], 2: [19, 822, 575, 664], 3: [19, 822, 575, 619], 4: [19, 822, 575, 619], 5: [19, 822, 575, 619], 6: [19, 822, 575, 619], 7: [19, 822, 575, 664], 8: [19, 822, 575, 664]}


In [56]:
enhanced_cropped_images= []
for i,page_image in enumerate(pages_images):
    cropped_cell_image = page_image.crop((head_coordinates_int[i][0], 0, page_image.width,(page_image.height/100)+head_coordinates_int[i][1])) #x1,y2,x2,y1
    cropped_cell_image = np.asarray(cropped_cell_image)
    gray_image = cv2.cvtColor(cropped_cell_image, cv2.COLOR_BGR2GRAY)

    min_val, max_val, _, _ = cv2.minMaxLoc(gray_image)

    print(f"Minimum Intensity: {min_val}")
    print(f"Maximum Intensity: {max_val}")

    # Get the minimum and maximum intensity values
    min_val = np.min(gray_image)
    max_val = np.max(gray_image)

    print(f"Minimum Intensity: {min_val}")
    print(f"Maximum Intensity: {max_val}")


    # Define a threshold value dynamically (e.g., mid-point)
    threshold = (min_val + max_val) / 2

    # Apply thresholding
    _, binary_image = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)
    enhanced_cropped_images.append(binary_image)
 

Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255
Minimum Intensity: 0.0
Maximum Intensity: 255.0
Minimum Intensity: 0
Maximum Intensity: 255


In [57]:
# # Perform OCR on the cropped cell
 # ocr_result = pytesseract.image_to_string(cropped_cell_image, lang="ara",config='--oem 3 --psm 6',)  # Add 'ara' for Arabic support
 # print(ocr_result, "\n###############################################\n")
 # # Save the cropped image (optional, for debugging purposes)
 # cropped_cell_image.save(fr".\cropped_images\{i}.png")
 # # Define the string you want to write to the file
 # # Open a file in write mode. If the file does not exist, it will be created.
 # with open(fr".\cropped_images\{i}.txt", "w") as file:
 #     # Write the string to the file
 #     file.write(ocr_result)

def extract_text_with_psm(image, lang='ara'):
    """
    Run Tesseract with different PSM values and return the best result.
    """
    # List of PSM modes to test
    psm_modes = [3, 4, 6, 7, 11, 12, 13]
    best_text = ""
    best_psm = None
    highest_confidence = 0
    
    # Load the image
    
    for psm in psm_modes:
        # Configure Tesseract with the current PSM
        config = f"--psm {psm} -l {lang}"
        data = pytesseract.image_to_data(image, config=config, output_type=Output.DICT)
        
        # Extract confidence scores and calculate average confidence
        conf = [conf for conf in data['conf'] if isinstance(conf, int) or conf.isdigit()]
        conf = list(map(int, conf))  # Ensure all confidence values are integers
        avg_confidence = sum(conf) / len(conf) if conf else 0
        
        # Get the extracted text
        text = pytesseract.image_to_string(image, config=config)
        
        # Update the best result if this PSM is better
        if avg_confidence > highest_confidence:
            highest_confidence = avg_confidence
            best_text = text
            best_psm = psm
    
    return best_psm, best_text, highest_confidence




In [59]:
# Usage
for i,enhanced_image in enumerate(enhanced_cropped_images):
    best_psm, best_text, confidence = extract_text_with_psm(enhanced_image)

    print(f"Best PSM: {best_psm}")
    print(f"Confidence: {confidence}")
    print("Extracted Text:")
    print(best_text, "\n###############################################\n")
    # Save the cropped image (optional, for debugging purposes)
    cv2.imwrite(fr'.\cropped_images\{i}.png',enhanced_image)
    with open(fr".\cropped_images\{i}.txt", "w") as file:
     # Write the string to the file
     file.write(best_text)
    

Best PSM: 6
Confidence: 64.94782608695652
Extracted Text:
٧٥٢٥101٢٦ 28-12-2023 2

تيابةشرق القاهرة الكلية لشنون الأسرة موقع الطباعة: جهجبر

نيابة القاهرة الجديدة الجزنية لشنون الأسرة ، .
أجندة يومية جنلسات َك

يومية جلسة يوم الاثنين الموافق 2024/12/16 بدائرة 19 انموذج رقم 42 "شنون أسرة")

برناسة القاضي نزار عبدالفتاح رئيس المحكمة

وعضوية كل من محمود دياب و محمد الشعراوي القاضيين

وبحضور كريم حسن عضو الدائرة الرابعم وبحضور ضياء الدين الاحمداوي عضو الدائرة الخامس

وبحضور أحمد مدحت نور الدين أحمد عضو النيابة

وبحضور الخبيرين أميره عاطف عبدالرازق خبير اجتماعي مني فايز أبوالوفا محمد خبير نفسي

وبحضور ناصر سامي عطيه خليل سكرتير الجلسة
 
###############################################

Best PSM: 4
Confidence: 63.56363636363636
Extracted Text:
‎٧٥٣٥101٨ 28-1 2-2023 2‏
تيابةشرق القاهرة الكلية لشنون الأسرة موقع الطباعة:
نيابة القاهرة الجديدة الجزئية لشنون الأسرة

‏أجندة يومية جلسات
يومية جلسة يوم الاثنين الموافق 2024/01/01 بدائرة 19 الموذج رقم 42 "شنون أسرة")

‏برناسة القاضي أحمد عبدالعزيز محمد 